In [1]:
import cifar10_input
import numpy as np
import tensorflow as tf
import time
import os

In [2]:
cifar10_input.maybe_download_and_extract()

In [3]:
# Hyperparams
# Architecture
N_HIDDEN_1 = 256
N_HIDDEN_2 = 256
ETA = 0.001
EPOCHS = 1000
BATCH_SIZE = 128
DISPLAY_STEP = 1

In [4]:
def inputs(eval_data=True):
    data_dir = os.path.join('data/cifar10_data', 'cifar-10-batches-bin')
    return cifar10_input.inputs(
        eval_data=eval_data, data_dir=data_dir, batch_size=BATCH_SIZE)

In [5]:
def distorted_inputs():
    data_dir = os.path.join('data/cifar10_data', 'cifar-10-batches-bin')
    return cifar10_input.distorted_inputs(data_dir=data_dir, 
                                          batch_size=BATCH_SIZE)

In [17]:
def filter_summary(V, weight_shape):
    ix = weight_shape[0] # ??? not used
    iy = weight_shape[1] # ???
    cx, cy = 8, 8        # ???
    V_T = tf.transpose(V, (3, 0, 1, 2)) # magic numbers!
    tf.summary.image('filters', V_T, max_outputs=64)

In [7]:
def conv2d(input, weight_shape, bias_shape, visualize=False):
    incoming = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(
        stddev=(2. / incoming) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable('W', weight_shape, initializer=weight_init)
    b = tf.get_variable('b', bias_shape, initializer=bias_init)
    if visualize: filter_summary(W, weight_shape)
    return tf.nn.relu(tf.nn.bias_add(
        tf.nn.conv2d(
            input, W, strides=[1, 1, 1, 1], padding='SAME'), 
        b))

In [8]:
def max_pool(input, k=2):
    return tf.nn.max_pool(
        input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [9]:
def layer(input, weight_shape, bias_shape):
    weight_init = tf.random_normal_initializer(
        stddev=(2. / weight_shape[0]) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable('W', weight_shape, initializer=weight_init)
    b = tf.get_variable('b', bias_shape, initializer=bias_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

In [10]:
def inference(x, keep_prob):
    with tf.variable_scope('conv_1'):
        conv_1 = conv2d(x, [5, 5, 3, 64], [64], visualize=True)
        pool_1 = max_pool(conv_1)
    with tf.variable_scope('conv_2'):
        conv_2 = conv2d(pool_1, [5, 5, 64, 64], [64])
        pool_2 = max_pool(conv_2)
    with tf.variable_scope('fc_1'): # fully connected
        dim = 1
        for d in pool_2.get_shape()[1:].as_list():
            dim *= d
        pool_2_flat = tf.reshape(pool_2, [-1, dim])
        fc_1 = layer(pool_2_flat, [dim, 384], [384])
        
        # Apply dropout
        fc_1_drop = tf.nn.dropout(fc_1, keep_prob)
    with tf.variable_scope('fc_2'):
        fc_2 = layer(fc_1_drop, [384, 192], [192])
        
        # Apply dropout
        fc_2_drop = tf.nn.dropout(fc_2, keep_prob)
    with tf.variable_scope('output'):
        output = layer(fc_2_drop, [192, 10], [10])
    return output

In [21]:
def loss(output, y):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=output, labels=tf.cast(y, tf.int64))
    loss = tf.reduce_mean(xentropy)
    return loss

In [24]:
def training(cost, global_step):
    tf.summary.scalar('cost', cost)
    optimizer = tf.train.AdamOptimizer(ETA)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [32]:
def evaluate(output, y):
    correct_pred = tf.equal(tf.cast(tf.argmax(output, 1), dtype=tf.int32),
                            y)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar('validation_error', 1. - accuracy)
    return accuracy

In [35]:
with tf.device('/gpu:0'):
    with tf.Graph().as_default():
        with tf.variable_scope('cifar_conv_model'):
            x = tf.placeholder('float', [None, 24, 24, 3])
            y = tf.placeholder('int32', [None])
            keep_prob = tf.placeholder(tf.float32)
            distorted_images, distorted_labels = distorted_inputs()
            val_images, val_labels = inputs()
            output = inference(x, keep_prob)
            cost = loss(output, y)
            global_step = tf.Variable(
                0, name='global_step', trainable=False)
            train_op = training(cost, global_step)
            eval_op = evaluate(output, y)
            summary_op = tf.summary.merge_all()
            saver = tf.train.Saver()
            sess = tf.Session()
            summary_writer = tf.summary.FileWriter(
                'conv_cifar_logs', graph=sess.graph)
            init_op = tf.global_variables_initializer()
            sess.run(init_op)
            tf.train.start_queue_runners(sess=sess)
            
            # Train
            for epoch in range(EPOCHS):
                avg_cost = 0.
                total_batches = int(
                    cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / 
                    BATCH_SIZE)
                
                # Loop over batches
                for i in range(total_batches):
                    # Fit to batch
                    train_x, train_y = sess.run([distorted_images, 
                                                 distorted_labels])
                    _, new_cost = sess.run(
                        [train_op, cost], 
                        feed_dict={x: train_x, y: train_y, keep_prob: 0.5})
                    
                    # Compute avg loss
                    avg_cost += new_cost / total_batches
                    
                # Display logs per epoch step
                if epoch % DISPLAY_STEP == 0:
                    val_x, val_y = sess.run([val_images, val_labels])
                    accuracy = sess.run(
                        eval_op, 
                        feed_dict={x: val_x, y: val_y, keep_prob: 1.})
                    print('Epoch: %04d\tCost: %.6f\tValidation error: %.6f'
                          %(epoch + 1, avg_cost, 1 - accuracy))
                    summary_str = sess.run(
                        summary_op, 
                        feed_dict={x: train_x, y: train_y, keep_prob: 1.})
                    summary_writer.add_summary(summary_str, 
                                               sess.run(global_step))
                    saver.save(sess, 
                               'conv_cifar_logs/model-checkpoint', 
                               global_step=global_step)
            print('Optimization finished!')
            
            val_x, val_y = sess.run([val_images, val_labels])
            accuracy = sess.run(
                eval_op, feed_dict={x: val_x, y: val_y, keep_prob: 1.})
            print('Test Accuracy:', accuracy)
                    

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
Epoch: 0001	Cost: 2.027943	Validation error: 0.539062
Epoch: 0002	Cost: 1.673000	Validation error: 0.554688
Epoch: 0003	Cost: 1.528226	Validation error: 0.406250
Epoch: 0004	Cost: 1.419284	Validation error: 0.398438
Epoch: 0005	Cost: 1.351768	Validation error: 0.476562
Epoch: 0006	Cost: 1.319155	Validation error: 0.335938
Epoch: 0007	Cost: 1.264800	Validation error: 0.367188
Epoch: 0008	Cost: 1.232235	Validation error: 0.343750
Epoch: 0009	Cost: 1.207253	Validation error: 0.320312
Epoch: 0010	Cost: 1.173475	Validation error: 0.367188
Epoch: 0011	Cost: 1.157171	Validation error: 0.398438
Epoch: 0012	Cost: 1.137940	Validation error: 0.273438
Epoch: 0013	Cost: 1.122789	Validation error: 0.359375
Epoch: 0014	Cost: 1.099199	Validation error: 0.273438
Epoch: 0015	Cost: 1.081709	Validation error: 0.273438
Epoch: 0016	Cost: 1.071992	Validation error: 0.273438
Epoch: 0017	Cost: 1.050765	Validation erro

KeyboardInterrupt: 